# AutoML Classification experiment using Remote-AML-Compute and AML-Datasets
## Data: IBM Employee Attrition dataset loaded from Azure ML Dataset

##  Get Azure ML Workspace to use

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

In [ ]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

# Get Workspace defined in by default config.json file
ws = Workspace.from_config()

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")

ws = Workspace(subscription_id="3c8972d9-f541-46b2-b70b-d81baba3595d",
                  resource_group="amlworkshop",
                  workspace_name="amlworkshop",
                  auth=interactive_auth)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Load data from Azure ML Datasets 
Pandas DataFrame only used to check out the data

In [ ]:
# Load Data
aml_dataset = ws.datasets['IBM-Employee-Attrition']

# Use Pandas DataFrame just to sneak peak some data and schema
full_df = aml_dataset.to_pandas_dataframe()
# .to_pandas_dataframe().dropna()
full_df.head(5)

In [ ]:
# Use Pandas DataFrame just to investigate the dataset's schema and info
full_df.describe()

## Clean up the initial dataset 
#### (Using AML Tabular Dataset .drop_columns() method )

In [ ]:
# Dropping Employee count as all values are 1 and hence attrition is independent of this feature
# Dropping Employee Number since it is merely an identifier
# Dropping Over18 since all employees are over 18
# Dropping StandardHours Since all values are 80
  
aml_dataset = aml_dataset.drop_columns(['EmployeeCount','EmployeeNumber','Over18','StandardHours'])

full_df = aml_dataset.to_pandas_dataframe()
full_df.describe()

## Split original AML Tabular Dataset in two test/train AML Tabular Datasets (using AML DS function)

In [ ]:
# Split using Azure Tabular Datasets (Better for Remote Compute)
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#random-split-percentage--seed-none-

train_dataset, test_dataset = aml_dataset.random_split(0.9, seed=1)

# Use Pandas DF only to check the data
train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

print(train_dataset_df.describe())
# print(test_dataset_df.describe())

## List remote AML compute targets available

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(ws)

## Connect to Remote AML Compute (Existing AML cluster)

In [ ]:
# Define remote compute target to use
# Further docs on Remote Compute Target: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-remote

# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     # Method 1:
     aml_remote_compute = cts[amlcompute_cluster_name]
     # Method 2:
     # aml_remote_compute = ComputeTarget(ws, amlcompute_cluster_name)
    
if not found:
     print('Creating a new training cluster...')
     provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D13_V2", # for GPU, use "STANDARD_NC12"
                                                                 #vm_priority = 'lowpriority', # optional
                                                                 max_nodes = 20)
     # Create the cluster.
     aml_remote_compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)
    


In [ ]:
# For additional details of current AmlCompute status:
aml_remote_compute.get_status().serialize()

## List and select primary metric to drive the AutoML classification problem

In [ ]:
from azureml.train import automl

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

# We'll use 'accuracy' as primary metric (Closer to 1.00 is better)

## Define AutoML Experiment settings (With AML Remote Compute)

In [ ]:
import logging
import os
# You can provide additional settings as a **kwargs parameter for the AutoMLConfig object
# automl_settings = {
#     "whitelist_models": 'XGBoostClassifier'
# }

from azureml.train.automl import AutoMLConfig

project_folder = './automl'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='classification',
                             primary_metric='accuracy',
                             experiment_timeout_minutes=15,                            
                             training_data=train_dataset,
                             label_column_name="Attrition",
                             n_cross_validations=5,
                             # blacklist_models='XGBoostClassifier', 
                             # iteration_timeout_minutes=5,                                                    
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             # **automl_settings
                             )

# PaperCut?: Why is drop_column_names only supported by Time Series Forecast? - If used for classification, you get:
# drop_column_names= ['EmployeeCount','EmployeeNumber','Over18','StandardHours'], # Clean up dataset by dropping not needed columns
# WARNING - Received unrecognized parameter: drop_column_names ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours']
# In documentation it doesn't state that it is only supported for Forecast...:
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py

# Explanation of Settings: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings

# AutoMLConfig info on: 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig


## Run Experiment (on AML Remote Compute) with multiple child runs under the covers

In [ ]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "classif-automl-remote-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

## Explore results with Widget

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

### Measure Parent Run Time needed for the whole AutoML process 

In [ ]:
import time
import datetime as dt

run_details = run.get_details()

# Like: 2020-01-12T23:11:56.292703Z
end_time_utc_str = run_details['endTimeUtc'].split(".")[0]
start_time_utc_str = run_details['startTimeUtc'].split(".")[0]
timestamp_end = time.mktime(datetime.strptime(end_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())
timestamp_start = time.mktime(datetime.strptime(start_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())

parent_run_time = timestamp_end - timestamp_start
print('Run Timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (parent_run_time))

## Retrieve the 'Best Model' (Scikit-Learn model)

In [ ]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

## Make Predictions

### Extract X values (feature columns) from test dataset and convert to NumPi array for predicting 

In [ ]:
import pandas as pd

#Remove Label/y column
if 'Attrition' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('Attrition')

x_test_df = test_dataset_df

### Make the actual Predictions

In [ ]:
# Try the best model
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

In [ ]:
y_predictions.shape

### Calculate the Accuracy with Test Dataset (Not used for training)

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy:')
accuracy_score(y_test_df, y_predictions)